# Part II: Data Wrangling

For the data wrangling examples, we will use the cholesterol dataset introduced in Part I

In [ ]:
import pandas as pd
import numpy as np
chol = pd.read_csv('cholesterol.csv')

## Basic data manipulations

Filtering can simply be achieved by using a boolean array:

In [ ]:
chol[chol.Cholesterol < 5]

In [ ]:
chol_filtered = chol[chol.Time == 1]
chol_filtered

Column selection is done by using the names of the columns:

In [ ]:
chol_selection = chol_filtered[['ID', 'Cholesterol']]
chol_selection

Arrays also have a function for sorting:

In [ ]:
chol_ordered = chol_selection.sort_values(by='Cholesterol')
chol_ordered

All of this can equally be achieved simultaneously:

In [ ]:
chol.loc[chol.Time == 1, ['ID', 'Cholesterol']].sort_values(by='Cholesterol')

There are multiple ways to select columns in pandas. A single column can be addressed using `.`:

In [ ]:
chol.Cholesterol

Column names can be extracted from the pandas object using `.columns`, and then used for selecting columns:

In [ ]:
chol[chol.columns[0:3]]

In [ ]:
chol.iloc[:, chol.columns.str.startswith('M')]

Using `.loc`, one can use row and column indices:

In [ ]:
chol.loc[:, chol.columns[0:3]]

`.iloc` allows indexing by integers rather than names:

In [ ]:
chol.iloc[:, 0:3]

Columns can be dropped using `.drop`:

In [ ]:
chol.drop('Cholesterol', axis=1)

Boolean index arrays can be joined by logical operators:


In [ ]:
chol.iloc[:, chol.columns.str.contains('gar') | chol.columns.str.endswith('ol')]

Renaming columns returns a new DataFrame:

In [ ]:
chol_copy = chol.rename(columns={'Cholesterol' : 'Ch'})

... but most pandas operations can also be performed in place

In [ ]:
chol_copy.rename(columns={'Time' : 'T'}, inplace=True)
chol_copy

Add new columns (mutates the original dataframe):

In [ ]:
chol['Cholesterol_2'] = chol['Cholesterol'] - 5
chol

Drop column again:

In [ ]:
chol.drop('Cholesterol_2', axis=1, inplace=True)

Sorting rows of a DataFrame:

In [ ]:
chol_sorted = chol.sort_values(by='ID')
chol_sorted

We can also access the sorting order by calling `.index`

In [ ]:
chol_sorted.index

In [ ]:
chol.sort_values(by='Cholesterol', ascending=False)

In [ ]:
chol.sort_values(by=['ID', 'Cholesterol'], ascending=[True, False])

## Aggregating

Computing summaries:

In [ ]:
chol.mean()['Cholesterol']

Computing summaries by group:

In [ ]:
chol.groupby('Time')['Cholesterol'].mean()

Grouping by multiple columns creates a hierarchical index, so we have to use sort_index instead of sort_values:

In [ ]:
chol.groupby(['Time', 'Margarine'])['Cholesterol'].mean().sort_index(level=[1, 0])

We can use `lambda` expressions to apply vectorized functions:

In [ ]:
chol.apply(lambda x: len(x.unique()))

## Practical example

We recode `Time` and `Margarine` as categorical variables:

In [ ]:
chol.Time = chol.Time.astype('category')
chol.Margarine = chol.Margarine.astype('category')

We want to split the dataset such that we have different columns for the cholesterol measurements at the three different time points. One way to achieve this is to split the dataset across time and then join the three resulting DataFrames.

In [ ]:
chol1 = chol[chol.Time == 1].drop('Time', axis=1)
chol2 = chol[chol.Time == 2].drop('Time', axis=1)
chol3 = chol[chol.Time == 3].drop('Time', axis=1)

chol1.merge(chol2, on=['ID', 'Margarine']).merge(chol3, on=['ID', 'Margarine'])

A more natural way of doing this, however, is by manipulating hierarchical indices by unstacking an index level to a column level

In [ ]:
chol_spread = chol.set_index(['ID', 'Margarine', 'Time']).unstack()
chol_spread.columns = chol_spread.columns.set_levels(['T%d' % i for i in chol_spread.columns.levels[1]], level=1)
chol_spread

The unstacking can be reversed using the function `.stack`...

In [ ]:
chol_spread.stack()

...or by dropping the index altogether

In [ ]:
chol_spread.stack().reset_index()

Shifting values in pandas Series and DataFrames:

In [ ]:
pd.Series(np.arange(1, 5)).shift(1)

In [ ]:
pd.Series(np.arange(1, 5)).shift(2)

Use diff to calculate difference between consecutive values:

In [ ]:
pd.Series(np.arange(1, 5)).diff()

In [ ]:
pd.Series(np.arange(1, 5)).diff(periods=2)

Using these functions, we can now create a new column that contains the cholesterol difference from the last measurement:

In [ ]:
diff = chol.groupby('ID').apply(lambda g: g.sort_values(by='Time')['Cholesterol'].diff()).reset_index(0)['Cholesterol']
chol.loc[diff.index, 'Cholesterol_Diff'] = diff
chol.sort_values(by=['ID', 'Time'])